In [2]:
import streamlit as st
import pandas as pd
import folium
from folium.plugins import MarkerCluster, HeatMap
from streamlit_folium import st_folium

In [3]:



# Page title
st.set_page_config(page_title='Permit Data Exploration', page_icon='📊')
st.title('📊 Permit Data Exploration')

with st.expander('About this app'):
    st.markdown('**What can this app do?**')
    st.info('Interact with the permit dataset/Parquet files and explore each column.')
    st.markdown('**How to use the app?**')
    st.warning('This is a prototype, functionality will be added as we develop further.')

st.subheader('Explore Permit Data by Construction Type and State')

2024-05-31 21:22:52.755 
  command:

    streamlit run /home/vscode/.local/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [4]:

# File upload
uploaded_file = st.file_uploader("Upload your Parquet file", type=['parquet'])

if uploaded_file is not None:
    df = pd.read_parquet(uploaded_file)
else:
    df = pd.read_parquet('data/csv_reveal-gc-2020-50.parquet')

In [5]:

# Input widgets
## Construction Type selection
const_type_list = df.CONST_TYPE.unique()
const_type_selection = st.multiselect('Select Construction Types', const_type_list, const_type_list[:3])


In [6]:

## State selection
state_list = df.SITE_STATE.unique()
state_selection = st.multiselect('Select States', state_list, state_list[:3])

jurisdiction_list = df['SITE_JURIS'].unique().tolist()
jurisdiction_selection = st.multiselect('Select Jurisdictions', jurisdiction_list, jurisdiction_list[:3])


In [7]:

# Filter data based on selections
df_selection = df[
    df['CONST_TYPE'].isin(const_type_selection) &
    df['SITE_STATE'].isin(state_selection) &
    df['SITE_JURIS'].isin(jurisdiction_selection)
]


In [17]:
df_selection.head()

,PERMITID,PRJ_STATUS,CONST_TYPE,SITE_AREA,SITE_STATE,SITE_CNTY,SITE_JURIS,SITE_CITY,SITE_ZIP,SITE_ADDRS,...,IMAGEID,DOCPARSERID,EXTRACTED_DESCRIPTION,EXTRACTED_NUMBER,EXTRACTED_VALUATION,EXTRACTED_UNITS,EXTRACTED_NUMBER_OF_BUILDINGS,EXTRACTED_STATUS,IMPUTE_UNITS_FLAG,IMPUTE_VALUATION_FLAG
0,17855172,Approved Permit,"Res Rmdl, Addn, Int Fin",Connecticut,CT,Fairfield County,Bridgeport,Bridgeport,06606-1042,81 Waller Rd,...,9804939.0,quadParserPseudoImage,"Add 2nd bathroom, add laundry room, add 4th be...",-333777,50000,,NaN,Need-Info -jg/m,B,D
1,17858630,Approved Permit,Single Family Homes,Orange County,CA,Orange County,Irvine,Irvine,92618-1334,107 Landroute,...,9696565.0,quadParserPseudoImage,(E-PLAN) Montara Phase 13 Portola Springs. Tra...,00824568-RBP,,,NaN,issued,A-Single Family Home,B
2,17855104,Approved Permit,"Res Rmdl, Addn, Int Fin",Connecticut,CT,Fairfield County,Bridgeport,Bridgeport,06606-5419,111 Gem Ave,...,9804935.0,quadParserPseudoImage,"Siding, Windows, Bath & Kitchen Remodel, New H...",-370746,9500,,NaN,NEED INFO -jg/m,B,D
3,17855372,Approved Permit,Alternative Residential Energy,Connecticut,CT,Fairfield County,Bridgeport,Bridgeport,06610-1725,66 Rose St,...,9804956.0,quadParserPseudoImage,Roof reinforcing for solar panel installation,-555113,5000,,NaN,jg/m,A-No Dwelling Units,D
4,17824051,Approved Permit,"Res Rmdl, Addn, Int Fin",South Dakota,SD,Lawrence County,Lawrence County,Lead,57754-3754,20687 Roman Rd,...,9668562.0,quadParserPseudoImage,"640'RES,340'LOFT,96'DECKHOLES ONLY TILL TRUSS ...",00008862,,,NaN,,B,B


In [9]:

# Display DataFrame
st.dataframe(df_selection)
# Filter data based on selections
df_selection = df[df.CONST_TYPE.isin(const_type_selection) & df.SITE_STATE.isin(state_selection)]

# Display DataFrame
st.dataframe(df_selection)


DeltaGenerator()

In [10]:

# Summary Statistics
st.subheader('Summary Statistics')
st.write(df_selection.describe())

2024-05-31 21:22:53.257 Serialization of dataframe to Arrow table was unsuccessful due to: ("Could not convert Timestamp('2020-12-21 09:34:25.589895') with type Timestamp: tried to convert to int64", 'Conversion failed for column SITE_GEOCODE_DATE with type object'). Applying automatic fixes for column types to make the dataframe Arrow-compatible.


In [11]:

# Pivot table to aggregate data
reshaped_df = df_selection.pivot_table(index='SITE_STATE', columns='CONST_TYPE', values='PERMITID', aggfunc='count', fill_value=0)
reshaped_df = reshaped_df.sort_values(by='SITE_STATE', ascending=False)


In [12]:

# Display reshaped DataFrame
st.subheader('Aggregated Data by State and Construction Type')
st.dataframe(reshaped_df)


DeltaGenerator()

In [13]:

# Prepare data for chart
df_chart = reshaped_df.reset_index().melt(id_vars='SITE_STATE', var_name='CONST_TYPE', value_name='COUNT')


In [14]:

# Display chart
chart = alt.Chart(df_chart).mark_bar().encode(
    x=alt.X('SITE_STATE:N', title='State'),
    y=alt.Y('COUNT:Q', title='Permit Count'),
    color='CONST_TYPE:N'
).properties(height=320)
st.altair_chart(chart, use_container_width=True)

NameError: name 'alt' is not defined

In [ ]:

# Map visualization
st.subheader('Permit Locations Map')

DeltaGenerator()

In [ ]:

# Clean data for map visualization
df_selection_map = df_selection[df_selection['SITE_LAT'].apply(lambda x: str(x).replace('.', '', 1).isdigit())]
df_selection_map = df_selection_map[df_selection_map['SITE_LONG'].apply(lambda x: str(x).replace('.', '', 1).lstrip('-').isdigit())]
df_selection_map['SITE_LAT1'] = df_selection_map['SITE_LAT'].astype(float)
df_selection_map['SITE_LONG1'] = df_selection_map['SITE_LONG'].astype(float)


In [ ]:

# Create a Folium map centered on the average latitude and longitude
if not df_selection_map.empty:
    map_center = [df_selection_map['SITE_LAT1'].mean(), df_selection_map['SITE_LONG1'].mean()]
    m = folium.Map(location=map_center, zoom_start=5)

    # Add marker cluster to the map
    marker_cluster = MarkerCluster().add_to(m)

    # Add points to the map
    for idx, row in df_selection_map.iterrows():
        folium.Marker(
            location=[row['SITE_LAT1'], row['SITE_LONG1']],
            popup=row['PMT_VALUE']
        ).add_to(marker_cluster)

    # Call to render Folium map in Streamlit
    st_data = st_folium(m, width=800, height=500)

    # Display the map
    st.write('Permit Locations Map:')
    st.write(st_data)
else:
    st.write("No valid coordinates available for mapping.")